# 0. Import needed Libraries

In [ ]:
# the 3 main libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from eofs.xarray import Eof as eof
# librairies to do nicer plots...
from matplotlib.patches import Rectangle    # only to draw a rectangle

# 1. TD summary:

In [ ]:
data = xr.open_dataset('sst.mnmean.nc')

In [ ]:
data = data.sel(time=slice('1982','2013'))   # keep only "complete" years
data = data.sel(lat=slice(59.5, -59.5))      # get rid of data out of 60S-60N

In [ ]:
mask = xr.open_dataset('lsmask.nc')
mask = mask.sel(lat=slice(59.5, -59.5))   # reduce memory footprint: get rid of data out of 60S-60N
mask = mask.squeeze()                     # get rid of the time dimension in the variable mask.mask

In [ ]:
weights = np.cos( np.deg2rad(mask.lat) )
weights = mask*weights

# 2. linear trend:

Long term mean trend ([linear regression](http://xarray.pydata.org/en/stable/computation.html?highlight=polyval#fitting-polynomials) with [polyfit](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.polyfit.html?highlight=polyfit) and [polyval](http://xarray.pydata.org/en/stable/generated/xarray.polyval.html?highlight=polyval) methonds)

In [ ]:
linfit = data.sst.polyfit('time', 1)               # compute linear regression coefficients
linfit.polyfit_coefficients

In [ ]:
slope = linfit.polyfit_coefficients.isel(degree=0)   # Polynomial coefficients, highest power first
ns_century = 1.e9*3600.*24.*365.*100.                # scaling: nono-seconds to century
(slope*ns_century).where(mask.mask == 1.).plot(vmin=-5,vmax=5,cmap='RdBu_r')
plt.title('Trend Slope in deg/century')

Detrend SST

In [ ]:
trend = xr.polyval(coord=data.time, coeffs=linfit.polyfit_coefficients)   # SST trend
sst_detrend = data.sst - trend.values + data.sst.mean(dim='time')         # detrended SST

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(6,3), constrained_layout=True)
a = data.sst.groupby('time.year').mean(dim='time')      # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(ax=axes[0], label="org")
a = trend.groupby('time.year').mean(dim='time')         # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(ax=axes[0], label="trend")
axes[0].legend()
axes[0].set_title('With trend')
#
a = sst_detrend.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights.mask).mean(dim=('lon','lat')).plot(ax=axes[1], label="detrend")
axes[1].legend()
axes[1].set_title('Without trend')

Detrended interannual anomaly

In [ ]:
sstbymth = sst_detrend.groupby("time.month")
mthclim = sstbymth.mean("time")       # detrended climatological months (seasonal cycle)
sstanom = sstbymth - mthclim          # detrended interannual annomaly
sstanom = sstanom.rename('sstanom')   # change variable name

interannual anomaly (with/without trend)

In [ ]:
a = data.sst.weighted(weights.mask).mean(dim=('lon','lat'))
clim = a.groupby("time.month").mean("time")   # climatological months (seasonal cycle)
anom = a.groupby("time.month") - clim         # interannual annomaly
anom.plot(label="org")
#
sstanom.weighted(weights.mask).mean(dim=('lon','lat')).plot(label="detrended")
plt.legend()

# 3. SST Regression onto Nino3.4:

Interannual standard deviation, with [nino3.4](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/) box (5S-5N and 170-120W).

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5), constrained_layout=True)
sst_detrend.std(dim='time').where(mask.mask == 1.).plot(ax=axes[0],vmin=0,vmax=5,cmap='YlGnBu')
axes[0].set_title('SST STD')
sstanom.std(dim='time').where(mask.mask == 1.).plot(ax=axes[1],vmin=0,vmax=2,cmap='YlGnBu')
axes[1].set_title('SST anom STD')
# overplot nino3.4 rectangle (170W-120W, 5S-5N)
ax = plt.gca()                                                              # Get the current reference
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='r',fill=False)   # Create a Rectangle patch
ax.add_patch(rect)                                                          # Add the patch to the Axes

Equatorial hovmoller of the SST interannual anomaly

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10, 10))
wgteq = weights.sel(lat=slice(2.5,-2.5), lon=slice(100.5, 285.5))
#
ssteq = sst_detrend.sel(lat=slice(2.5,-2.5), lon=slice(100.5, 285.5))
ssteq.weighted(wgteq.mask).mean(dim='lat').plot(ax=axes[0])
#
ssteq = sstanom.sel(lat=slice(2.5,-2.5), lon=slice(100.5, 285.5))
ssteq.weighted(wgteq.mask).mean(dim='lat').plot(ax=axes[1])

Compute nino3.4 (5S-5N and 170-120W) index

In [ ]:
nino34 = sstanom.sel(lon=slice(360-169.5,360-120.5), lat=slice(4.5,-4.5))
weights_nino34 = weights.mask.sel(lon=slice(360-169.5,360-120.5), lat=slice(4.5,-4.5))
nino34_index = sstanom.weighted(weights_nino34).mean(dim=('lon','lat'))
nino34_index = nino34_index.rename('nino34')   # change variable name
nino34_index.plot()

SST anomalies regressed onto Nino3.4 SST

In [ ]:
sstanom.coords["time"] = (("time"), nino34_index.data)  # redefine 'time' coordinates with nino34 time series
linfit = sstanom.polyfit('time', 1)                     # compute the regression with this new "time" 
sstanom.coords["time"] = (("time"), data.time.data)     # put back original time
sstreg = linfit.polyfit_coefficients.isel(degree=0)
#
fig, axes = plt.subplots(1,2,figsize=(14, 5))
sstreg.where(mask.mask == 1.).plot(ax=axes[0])                              # plot the regression coefficient
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[0].add_patch(rect)                                                     # overlay 
axes[0].set_title('Regression coefficient')
sstcor = xr.corr(sstanom, nino34_index, dim='time')
sstcor.where(mask.mask == 1.).plot(ax=axes[1])
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[1].add_patch(rect)                                                     # overlay 
axes[1].set_title('Correlation')
axes[1].set_xlabel('lon')
axes[1].set_ylabel('lat')